
<h1><font color="teal">Sample Data Analysis - Automation Code (Genesis WS1)</font></h1>


In [ ]:

from datetime import date
print('Run date:')
print(str(date.today())) 


<h3><font color="purple">Import libraries and set working directory</font></h3>


In [ ]:

import os
import pandas as pd
import numpy as np
import itertools
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


In [ ]:
os.getcwd()

In [ ]:

os.chdir('C:\\Users\\kv23857\\Documents\\DAT\\ALL')

In [ ]:
filepath = os.getcwd()
print(filepath)



<h3><font color="blue">Enter Product Name (pos, tp, lns, cad, dep, dev)</font></h3>


In [ ]:
prodname = 'dep'


<h3><font color="purple">Function to set TRL filename, sheetnames, and sample dataset names based on given product name</font></h3>


In [ ]:

if prodname=='dep':
    trl_filename = 'Deposit_TRL_V15.xlsx'
    trl_data = ['Indicative Data - Deposits', 'Indicative-Extension', 'Deposit Collateral','Arrng Involved Party','Common Balance']
    sample_data = ['id', 'ide', 'col', 'ip', 'cb']
elif prodname == 'lns':
    trl_filename = 'Corporate_Loans_TRL_V16.xlsx'
    trl_data = ['Indicative Record','Arrangement Involved Parties','Arrangement Associations','Arrangement Rates','Installments','Arrng Loss Mitigation','Common Balance']
    sample_data = ['id','ip','arar','ar','ins','alm','cb']
elif prodname == 'pos':
    trl_filename = 'Security_Positions_Target_Layout_V15.xlsx'
    trl_data = ['Financial Market Position','Account Reference','Common Balance','Hedge']
    sample_data = ['id','ar','cb','hdge']
elif prodname == 'tp':
    trl_filename = 'Trade_Products_TRL_V15.xlsx'
    trl_data = ['Indicative','Arrangement Associations','Arrangement Involved Parties','Arrangement Rates','Arrangement Participation','Arrangement Alt Id','Arrng Modification','Common Balance']
    sample_data = ['id','aa','ip','ar','ap','alt','am','cb']
elif prodname == 'dev':
    trl_filename = 'OTC_Derivative_TRL_V15.xlsx'
    trl_data = ['Core OTC Deriv Arr','OTC Deriv Leg','Account Reference Data','Arr Assoc','Common Balance','Incr OTC Deriv Arr','Incr OTC Payment Schd','Incr OTC Option Exercise Schd']
    sample_data = ['ocda','odl','ar','oaa','ocb','oida','ps','oioe']
else:
    print("Error - incorrect product name")


In [ ]:

trl = pd.ExcelFile(trl_filename)
trl.sheet_names


<h3><font color="purple">Create list of all sample data filenames within folder in given path</font></h3>


In [ ]:
list_dat = os.listdir(filepath)
list_dat = [x for x in list_dat if '.dat' in x]
print(len(list_dat))

<h3><font color="purple">Function to convert all input data cols to string/object type while importing to python</font></h3>

In [ ]:
#Converts all columns to a string or object datatype while importing .dat file to Python
class StringConverter(dict):
    def __contains__(self, item):
        return True

    def __getitem__(self, item):
        return str

    def get(self, default=None):
        return str



<h3><font color="purple">Loop through sample data files for given product and concatenate data into one master dataframe</font></h3>


In [ ]:

# create empty dataframe
df_master = pd.DataFrame()
# loop through each dat files 
for each_dat in list_dat:
    if each_dat.split('_')[2] == prodname:
        temp_path_dat = os.path.join(filepath, each_dat)
        # temporary dataframe
        col_names = list(range(0,500)) 
        df_temp = pd.read_csv(temp_path_dat,sep="|",header=None,converters=StringConverter(),names=col_names,encoding='unicode_escape')

        # add a column with the datetime from filename
        df_temp['Filename'] = each_dat
        df_temp['CSI ID'] = each_dat.split('_')[1]
        df_temp['Product'] = each_dat.split('_')[2]
        df_temp['Dataset'] = each_dat.split('_')[3]
        df_temp['Country'] = each_dat.split('_')[5]
        dat = each_dat.split('_')[-1]
        df_temp['Date/Timestamp'] = dat.split('.')[0]
        # concatenate dataframes
        df_master = pd.concat([df_master, df_temp])

    # reset the dataframe index
    df_master = df_master.reset_index(drop=True)




<h3><font color="purple">Create dictionary of dataframes for each unique sample dataset</font></h3>


In [ ]:
unique_dataset = df_master.Dataset.unique()
df_dict = {name: df_master.loc[df_master['Dataset'] == name] for name in unique_dataset}

In [ ]:

#Display all dataset names
print(len(df_dict))
df_dict.keys()


<h3><font color="purple">Parse the TRL file for given sheetnames</font></h3>


In [ ]:
for sheet_name in trl.sheet_names:
    trl_dict = trl.parse(trl_data,dtype='object')

In [ ]:
print(len(trl_dict))
trl_dict.keys()

In [ ]:
trl_dict['Common Balance'].head(2)

<h3><font color="purple">Loop through TRL sheets to check for 'Business Attribute Name' and send to concatenated list.</font></h3>

In [ ]:
for x,y in trl_dict.items():
    attr_cols = [col for col in y.columns if 'Business Attribute Name' in col]
    print(x,attr_cols)
    y = y[~y['Business Attribute Name'].isnull()]
    trl_dict[x]= y

In [ ]:
attr_series = []
for k, v in trl_dict.items():
    temp = v['Business Attribute Name'].tolist()
    attr_series.append(temp)

In [ ]:
i = 0
while i < len(sample_data):
    attr_series[i].append(sample_data[i])
    i +=1

In [ ]:
itemDict = {attr[-1]: attr for attr in attr_series}


<h3><font color="purple">Assign TRL business attribute names as column headers for each sample dataset.</font></h3>


In [ ]:
new_df_dict = df_dict.copy()

In [ ]:
for key,val in new_df_dict.items():
    new_df_dict[key] = new_df_dict[key].reset_index(drop=True)  
    newcols = ['0']
    newlist = itemDict[key].copy()
    newcols.extend(newlist)
    newcols.extend(list(range(new_df_dict[key].shape[1]-len(newcols))))
    new_df_dict[key].columns = newcols  
    new_df_dict[key].drop(new_df_dict[key].iloc[:,len(newlist):-6], axis=1, inplace=True)

In [ ]:
for d,p in new_df_dict.items():
    p.rename(columns={'0': 'H_D_T',p.columns[-6]: 'Filename', p.columns[-5]: 'CSI ID',p.columns[-4]: 'Product',p.columns[-3]: 'Dataset',p.columns[-2]: 'Country',p.columns[-1]: 'Date/Timestamp'},inplace=True)
    

In [ ]:
new_df_dict['cb'].head()


<h3><font color="purple">Export sample datasets with assigned column headers and metadata details to csv files</font></h3>


In [ ]:
for i in new_df_dict: 
    new_df_dict[i].to_csv(prodname+'_sampledata_'+str(i)+'.csv',index=False)


<h3><font color="purple">Function to format each product TRL and create master dataframe (analysis document)</font></h3>


In [ ]:

def dep_trl_func(d):
    for k, i in d.items():
        global masterdf
        i['Dataset'] = k
        #checkcols = [check for check in i.columns if 'Remarks' in check]
        #print(k, ' : ',checkcols)
        matchers = ['Dataset','Physical','Business Attribute', 'Data Type & Length','Mandatory','Attribute Description','Expected','Default','CDE Name', 'Tracking','CDE', 'Remarks']
        matching = [s for s in i.columns if any(xs in s for xs in matchers)]
        #print(k,matching)
        x = i.columns[i.columns.str.contains('pdate')]
        i = i.drop(columns=x)
        i = i.drop(columns=[col for col in i if col not in matching])
        #print(k," - ", len(i.columns))
        #print(i.columns)
        i.columns = ['Element Name','Description','Data Type & Length', 'MCO','Remarks', 'Expected Domain Values', 'Default Values','CDE Tracking ID', 'CDE Name', 'Drop','Drop','Drop','Drop','CDE','Physical Name','Dataset']
        i = i.drop(columns = 'Drop')
        d[k]= i
        masterdf = pd.concat(d, axis=0,sort=False).reset_index()
    return masterdf                    



In [ ]:
if prodname == 'lns':
    print("Executing TRL cleanup function for Loans sample data...")
    lns_trl_func(trl_dict)
elif prodname == 'pos':
    print("Executing TRL cleanup function for Security Positions sample data...")
    pos_trl_func(trl_dict)
elif prodname == 'tp':
    print("Executing TRL cleanup function for Traded Products sample data ...")
    tp_trl_func(trl_dict)
elif prodname == 'dep':
    print("Executing TRL cleanup function for Deposits sample data...")
    dep_trl_func(trl_dict)
    unique_dataset = masterdf.level_0.unique()
    masterdf.level_0 = masterdf.level_0.apply(lambda x: sample_data[0] if unique_dataset[0] in x else sample_data[1] if unique_dataset[1] in x else sample_data[2] if unique_dataset[2] in x else sample_data[3] if unique_dataset[3] in x else sample_data[4] if unique_dataset[4] in x else x)    
    print(masterdf.keys())
elif prodname == 'dev':
    print("Executing TRL cleanup function for OTC sample data...")
    otc_trl_func(trl_dict)
    unique_dataset = masterdf.level_0.unique()
    masterdf.level_0 = masterdf.level_0.apply(lambda x: sample_data[0] if unique_dataset[0] in x else sample_data[1] if unique_dataset[1] in x else sample_data[2] if unique_dataset[2] in x else sample_data[3] if unique_dataset[3] in x else sample_data[4] if unique_dataset[4] in x else sample_data[5] if unique_dataset[5] in x else sample_data[6] if unique_dataset[6] in x else sample_data[7] if unique_dataset[7] in x else sample_data[8] if unique_dataset[8] in x else x)   
    print(masterdf.keys())
else:
    print("Error - incorrect product name")

In [ ]:
masterdf.drop(columns = 'level_1',inplace=True)

In [ ]:
masterdf.head(2)


<h3><font color="purple">Add sample data analysis columns to the master dataframe</font></h3>


In [ ]:
masterdf2 = masterdf[~masterdf['Element Name'].isnull()]

In [ ]:
masterdf2 = masterdf2.reindex(columns=[*masterdf2.columns.tolist(), 'Null/Not Null/Zero', 'Number of Records Analyzed', 'Sample Data File Date', 'Sample Data Source', 'Representative Sample Values','Sample Data Analyzed for Country','Filename', 'Potential Gap','Observations'])

In [ ]:
masterdf2.columns[masterdf2.isna().all()].tolist()


<h3><font color="purple">Send sample data to 'Representative Sample Values' column</font></h3>


In [ ]:
data1 = []
for j,k in new_df_dict.items():
    data = k.iloc[1].tolist()
    data1.append(data)

In [ ]:

new_list = sorted(data1, key=lambda x:sample_data.index(x[-3]))

In [ ]:

for qwe in new_list:
    if qwe[0] == 'T':
        qwe[1]= 'No Data Available'
        qwe[2]= ''
        qwe[3]= ''
        qwe[4]= ''

In [ ]:

result_list = [i[1:-6] for i in new_list]

In [ ]:

append_list = [item for sublist in result_list for item in sublist]

In [ ]:

masterdf2['Representative Sample Values'] = append_list


<h3><font color="purple">Assign values to 'Null/Not Null/Zero' column based on sample values</font></h3>


In [ ]:

masterdf2['Representative Sample Values']=masterdf2['Representative Sample Values'].replace(r'^\s*$', np.nan, regex=True).fillna('--')
masterdf2['Null/Not Null/Zero'] = pd.np.where(masterdf2['Representative Sample Values'].str.contains("--"),"Null", pd.np.where(masterdf2['Representative Sample Values'].str.strip()==("0"),"Zero","Not Null"))

In [ ]:

masterdf2[['Null/Not Null/Zero','Representative Sample Values']].head(10)


<h3><font color="darkblue">Assign values to 'Sample Data File Date' column based on sample values</font></h3>


In [ ]:

check_list = [len(i) for i in result_list]
print(check_list)

In [ ]:

sum(check_list)

In [ ]:

datelist11=[]
for j,k in new_df_dict.items():
    datelist = k['Date/Timestamp'].unique().tolist()
    datasetlist = k.Dataset.unique().tolist()
    datelist1 = datelist + datasetlist
    datelist11.append(datelist1)

In [ ]:

#remove empty strings from nested list
# b = [[i for i in item if i != ''] for item in datelist11]
# c = [item for item in b if item != []]



In [ ]:

print(datelist11)

In [ ]:

datelist2 = sorted(datelist11, key=lambda w:sample_data.index(w[-1]))

In [ ]:

print(datelist2)

In [ ]:

finaldd=[]
for i in range(len(check_list)):
    date=[el[0] for el in datelist2][i]
    dd= (np.repeat(date,check_list[i])).tolist()
    i +=1
    lyst = [x for x in dd]
    finaldd.append(lyst)
    finaldd2 = [item for sublist in finaldd for item in sublist]

In [ ]:

len(finaldd2)

In [ ]:

masterdf2['Sample Data File Date'] = finaldd2


<h3><font color="darkblue">Assign values to 'Filename', 'Sample Data Analyzed for Country', 'Number of Records Analyzed', 'Sample Data File Date', and 'Sample Data Source' columns</font></h3>


In [ ]:

filelist1=[]
for j,k in new_df_dict.items():
    filelist = k.Filename.unique().tolist()
    countrylist = k.Country.unique().tolist()
    datasetlist = k.Dataset.unique().tolist()
    newlist = filelist + countrylist + datasetlist
    filelist1.append(newlist)

In [ ]:

filelist2 = sorted(filelist1, key=lambda w:sample_data.index(w[-1]))



In [ ]:

filelist2



In [ ]:

finalff=[]
for i in range(len(check_list)):
    file=[el[0] for el in filelist2][i]
    ff= (np.repeat(file,check_list[i])).tolist()
    i +=1
    lyst = [x for x in ff]
    finalff.append(lyst)
    finalff2 = [item for sublist in finalff for item in sublist]

In [ ]:

len(finalff2)

In [ ]:

masterdf2['Filename'] = finalff2

In [ ]:
finalcc=[]
for i in range(len(check_list)):
    country=[el[1] for el in filelist2][i]
    cc= (np.repeat(country,check_list[i])).tolist()
    i +=1
    lyst = [x for x in cc]
    finalcc.append(lyst)
    finalcc2 = [item for sublist in finalcc for item in sublist]

In [ ]:
len(finalcc2)

In [ ]:
masterdf2['Sample Data Analyzed for Country'] = finalcc2

In [ ]:
lengthlist =[]
for j,k in new_df_dict.items():
    chk = len(k['H_D_T'])
    chk = [chk-2]
    datasetlist = k.Dataset.unique().tolist()
    lenlist = chk+datasetlist
    lengthlist.append(lenlist)

In [ ]:
print(lengthlist)

In [ ]:
lengthlist2 = sorted(lengthlist, key=lambda w:sample_data.index(w[-1]))

In [ ]:
print(lengthlist2)

In [ ]:
finalz=[]
for i in range(len(check_list)):
    global lyst
    length = [el[0] for el in lengthlist2][i]
    ll= (np.repeat(length,check_list[i])).tolist()
    i +=1
    lyst = [x for x in ll]
    finalz.extend(lyst)

In [ ]:
masterdf2['Number of Records Analyzed'] =finalz

In [ ]:
masterdf2['Sample Data Source']='Production' 

In [ ]:
masterdf2.to_csv(prodname + '_TRL_Analysis' +'.csv',index=False)


<h3><font color="darkblue">WIP - Potential Gaps and Observations</font></h3>


In [ ]:
masterdf3 = masterdf2.copy()

In [ ]:
masterdf3.head(2)

In [ ]:
masterdf3['MCO']=masterdf3['MCO'].fillna('-')

In [ ]:
masterdf3['Potential Gap'] = np.where(masterdf3['MCO'].str.contains('M') & masterdf3['Null/Not Null/Zero']=="Null", 'Y', 'N')

In [ ]:
masterdf3[['Null/Not Null/Zero','MCO','Potential Gap']].head(10)

In [ ]:
masterdf3.to_csv(prodname + '_TRL_Analysis_updates' +'.csv',index=False)